In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

In [2]:
'''
NOTES:

   Inverse Transform => 
     https://stackoverflow.com/questions/49330195/how-to-use-inverse-transform-in-minmaxscaler-for-a-column-in-a-matrix
'''
DATA_PATH = './data/'

In [3]:
USD_TRY = pd.read_csv(f"{DATA_PATH}USDTRY=X.csv")
USD_TRY.columns = [str(col) + '_USD_TRY' if str(col) != "Date" else str(col) for col in USD_TRY.columns]

XU_100 = pd.read_csv(f"{DATA_PATH}XU100.IS.csv")
XU_100.columns = [str(col) + '_XU_100' if str(col) != "Date" else str(col) for col in XU_100.columns]

TUPRS = pd.read_csv(f"{DATA_PATH}TUPRS.IS.csv")
TUPRS.columns = [str(col) + '_TUPRS' if str(col) != "Date" else str(col) for col in TUPRS.columns]

THY = pd.read_csv(f"{DATA_PATH}THYAO.IS.csv")
THY.columns = [str(col) + '_THY' if str(col) != "Date" else str(col) for col in THY.columns]

ASELS = pd.read_csv(f"{DATA_PATH}ASELS.IS.csv")
ASELS.columns = [str(col) + '_ASELS' if str(col) != "Date" else str(col) for col in ASELS.columns]

In [4]:
USD_TRY.head()

,Date,Open_USD_TRY,High_USD_TRY,Low_USD_TRY,Close_USD_TRY,Adj Close_USD_TRY,Volume_USD_TRY
0,2005-01-03,1.3455,1.3455,1.3368,1.3440,1.3440,0.0
1,2005-01-04,1.3368,1.3510,1.3368,1.3510,1.3510,0.0
2,2005-01-05,1.3545,1.3865,1.3545,1.3740,1.3740,0.0
3,2005-01-06,1.3780,1.4058,1.3780,1.3925,1.3925,0.0
4,2005-01-07,1.3870,1.3940,1.3615,1.3940,1.3940,0.0


In [5]:
XU_100.head()

,Date,Open_XU_100,High_XU_100,Low_XU_100,Close_XU_100,Adj Close_XU_100,Volume_XU_100
0,1997-07-01,1857.0,1864.0,1826.0,1851.0,1851.0,2342600.0
1,1997-07-02,1851.0,1877.0,1849.0,1873.0,1873.0,2946200.0
2,1997-07-03,1873.0,1916.0,1871.0,1912.0,1912.0,3660300.0
3,1997-07-04,1912.0,1936.0,1897.0,1918.0,1918.0,3411100.0
4,1997-07-07,1918.0,1963.0,1918.0,1953.0,1953.0,3045500.0


In [6]:
TUPRS.head()

,Date,Open_TUPRS,High_TUPRS,Low_TUPRS,Close_TUPRS,Adj Close_TUPRS,Volume_TUPRS
0,2000-05-10,11.343283,11.492537,11.044776,11.343283,2.129507e+09,1728772.0
1,2000-05-11,10.746268,11.194029,10.597014,10.746268,2.017427e+09,888562.0
2,2000-05-12,11.044776,11.343283,10.895522,11.044776,2.073467e+09,816842.0
3,2000-05-15,10.597014,11.194029,10.597014,10.597014,1.989408e+09,434831.0
4,2000-05-16,10.597014,10.746268,10.298507,10.597014,1.989408e+09,580223.0


In [7]:
THY.head()

,Date,Open_THY,High_THY,Low_THY,Close_THY,Adj Close_THY,Volume_THY
0,2000-05-10,0.001870,0.001870,0.001806,0.001838,0.001694,212630653.0
1,2000-05-11,0.001806,0.001806,0.001711,0.001775,0.001636,211439905.0
2,2000-05-12,0.001806,0.001806,0.001775,0.001806,0.001664,123850733.0
3,2000-05-15,0.001775,0.001775,0.001711,0.001711,0.001577,145015837.0
4,2000-05-16,0.001743,0.001743,0.001680,0.001743,0.001606,150023501.0


In [8]:
ASELS.head()

,Date,Open_ASELS,High_ASELS,Low_ASELS,Close_ASELS,Adj Close_ASELS,Volume_ASELS
0,2000-05-10,9.409108,9.526721,9.173880,9.409108,7.102494,44552.0
1,2000-05-11,9.173880,9.173880,9.056266,9.173880,6.924932,20159.0
2,2000-05-12,9.056266,9.409108,9.056266,9.056266,6.836151,24227.0
3,2000-05-15,8.821038,9.056266,8.703424,8.821038,6.658588,27102.0
4,2000-05-16,8.703424,8.821038,8.468197,8.703424,6.569807,44044.0


#### Tarihlerin başlangıç ve bitiş değerleri aynı olmalı.

In [9]:
print([USD_TRY["Date"][0], USD_TRY["Date"].iloc[-1]], 
      [XU_100["Date"][0], XU_100["Date"].iloc[-1]], 
      [TUPRS["Date"][0], TUPRS["Date"].iloc[-1]], 
      [THY["Date"][0], THY["Date"].iloc[-1]], 
      [ASELS["Date"][0], ASELS["Date"].iloc[-1]])

# Başlangıç Tarihi => 2005-01-03
# Bitiş Tarihi => 2021-05-05

['2005-01-03', '2021-05-12'] ['1997-07-01', '2021-05-05'] ['2000-05-10', '2021-05-12'] ['2000-05-10', '2021-05-12'] ['2000-05-10', '2021-05-12']


In [10]:
# Filtering dates before 2005-01-03 and after 2021-05-05
USD_TRY = USD_TRY.loc[(USD_TRY["Date"] >= '2005-01-03') & (USD_TRY["Date"] <= '2021-05-05')]
XU_100 = XU_100.loc[(XU_100["Date"] >= '2005-01-03') & (XU_100["Date"] <= '2021-05-05')]
TUPRS = TUPRS.loc[(TUPRS["Date"] >= '2005-01-03') & (TUPRS["Date"] <= '2021-05-05')]
THY = THY.loc[(THY["Date"] >= '2005-01-03') & (THY["Date"] <= '2021-05-05')]
ASELS = ASELS.loc[(ASELS["Date"] >= '2005-01-03') & (ASELS["Date"] <= '2021-05-05')]

In [11]:
USD_TRY

,Date,Open_USD_TRY,High_USD_TRY,Low_USD_TRY,Close_USD_TRY,Adj Close_USD_TRY,Volume_USD_TRY
0,2005-01-03,1.34550,1.34550,1.33680,1.34400,1.34400,0.0
1,2005-01-04,1.33680,1.35100,1.33680,1.35100,1.35100,0.0
2,2005-01-05,1.35450,1.38650,1.35450,1.37400,1.37400,0.0
3,2005-01-06,1.37800,1.40580,1.37800,1.39250,1.39250,0.0
4,2005-01-07,1.38700,1.39400,1.36150,1.39400,1.39400,0.0
...,...,...,...,...,...,...,...
4258,2021-04-29,8.19990,8.24920,8.14060,8.18489,8.18489,0.0
4259,2021-04-30,8.20330,8.31147,8.18517,8.18488,8.18488,0.0
4260,2021-05-03,8.28770,8.33030,8.24477,8.27455,8.27455,0.0
4261,2021-05-04,8.24890,8.34394,8.24491,8.25900,8.25900,0.0


In [12]:
XU_100

,Date,Open_XU_100,High_XU_100,Low_XU_100,Close_XU_100,Adj Close_XU_100,Volume_XU_100
1959,2005-01-03,24971.699219,25508.099609,24918.199219,25445.199219,25445.199219,2.278007e+08
1960,2005-01-04,25445.199219,25602.199219,24992.800781,25042.000000,25042.000000,2.688344e+08
1961,2005-01-05,25042.000000,25042.000000,24264.000000,24422.900391,24422.900391,2.062249e+08
1962,2005-01-06,24422.900391,24588.400391,24237.699219,24561.900391,24561.900391,1.618173e+08
1963,2005-01-07,24561.900391,25308.300781,24481.099609,25308.300781,25308.300781,3.037296e+08
...,...,...,...,...,...,...,...
6176,2021-04-29,1390.900024,1406.500000,1390.900024,1401.500000,1401.500000,1.549958e+09
6177,2021-04-30,1399.099976,1405.900024,1396.500000,1397.800049,1397.800049,1.838904e+09
6178,2021-05-03,1400.199951,1421.199951,1400.199951,1421.199951,1421.199951,1.946216e+09
6179,2021-05-04,1423.199951,1428.099976,1411.000000,1416.300049,1416.300049,2.277663e+09


In [13]:
TUPRS

,Date,Open_TUPRS,High_TUPRS,Low_TUPRS,Close_TUPRS,Adj Close_TUPRS,Volume_TUPRS
1213,2005-01-03,13.500000,13.700000,13.500000,13.500000,3.237362,173120.0
1214,2005-01-04,13.400000,13.700000,13.300000,13.400000,3.213381,286043.0
1215,2005-01-05,12.700000,13.100000,12.600000,12.700000,3.045517,332262.0
1216,2005-01-06,12.200000,12.800000,12.000000,12.200000,2.925616,1126627.0
1217,2005-01-07,12.700000,13.600000,12.500000,12.700000,3.045517,3214498.0
...,...,...,...,...,...,...,...
5430,2021-04-29,87.500000,88.449997,87.250000,87.949997,87.949997,2730392.0
5431,2021-04-30,88.050003,88.800003,87.500000,87.750000,87.750000,3485955.0
5432,2021-05-03,88.000000,90.849998,87.949997,90.849998,90.849998,5176702.0
5433,2021-05-04,90.900002,91.400002,89.699997,90.150002,90.150002,4046707.0


In [14]:
THY

,Date,Open_THY,High_THY,Low_THY,Close_THY,Adj Close_THY,Volume_THY
1213,2005-01-03,1.001774,1.014454,0.995433,1.001774,0.923266,3453812.0
1214,2005-01-04,0.989093,1.014454,0.989093,0.989093,0.911579,6309848.0
1215,2005-01-05,0.932030,0.970072,0.932030,0.932030,0.858988,10116362.0
1216,2005-01-06,0.944710,0.951051,0.932030,0.944710,0.870674,3446651.0
1217,2005-01-07,0.938370,0.951051,0.932030,0.938370,0.864831,13860537.0
...,...,...,...,...,...,...,...
5430,2021-04-29,12.050000,12.070000,11.890000,11.980000,11.980000,31262763.0
5431,2021-04-30,11.960000,12.330000,11.950000,12.230000,12.230000,77606295.0
5432,2021-05-03,12.330000,12.950000,12.300000,12.920000,12.920000,99132973.0
5433,2021-05-04,13.080000,13.090000,12.620000,12.620000,12.620000,103271085.0


In [15]:
ASELS

,Date,Open_ASELS,High_ASELS,Low_ASELS,Close_ASELS,Adj Close_ASELS,Volume_ASELS
1213,2005-01-03,0.17348,0.174950,0.169069,0.17348,0.130952,21728937.0
1214,2005-01-04,0.17642,0.179361,0.174950,0.17642,0.133171,26989950.0
1215,2005-01-05,0.17348,0.177890,0.172010,0.17348,0.130952,14049365.0
1216,2005-01-06,0.17642,0.176420,0.173480,0.17642,0.133171,12079665.0
1217,2005-01-07,0.17642,0.176420,0.173480,0.17642,0.133171,11591151.0
...,...,...,...,...,...,...,...
5430,2021-04-29,14.96000,15.210000,14.960000,15.02000,15.020000,14062484.0
5431,2021-04-30,15.05000,15.200000,14.950000,15.00000,15.000000,15692828.0
5432,2021-05-03,15.04000,15.140000,15.010000,15.11000,15.110000,11659676.0
5433,2021-05-04,15.14000,15.160000,14.910000,14.91000,14.910000,12450217.0


In [16]:
#pd.merge(pd.merge(pd.merge(THY, ASELS, on=["Date"], how="inner"), TUPRS, on=["Date"], how="inner"), XU_100, on=["Date"], how="inner")
X = pd.merge(XU_100[XU_100.columns[[0,1,2,3,4,6]]], USD_TRY[USD_TRY.columns[[0,1,2,3,4]]], on=["Date"], how="inner")
X

,Date,Open_XU_100,High_XU_100,Low_XU_100,Close_XU_100,Volume_XU_100,Open_USD_TRY,High_USD_TRY,Low_USD_TRY,Close_USD_TRY
0,2005-01-03,24971.699219,25508.099609,24918.199219,25445.199219,2.278007e+08,1.34550,1.34550,1.33680,1.34400
1,2005-01-04,25445.199219,25602.199219,24992.800781,25042.000000,2.688344e+08,1.33680,1.35100,1.33680,1.35100
2,2005-01-05,25042.000000,25042.000000,24264.000000,24422.900391,2.062249e+08,1.35450,1.38650,1.35450,1.37400
3,2005-01-06,24422.900391,24588.400391,24237.699219,24561.900391,1.618173e+08,1.37800,1.40580,1.37800,1.39250
4,2005-01-07,24561.900391,25308.300781,24481.099609,25308.300781,3.037296e+08,1.38700,1.39400,1.36150,1.39400
...,...,...,...,...,...,...,...,...,...,...
4216,2021-04-29,1390.900024,1406.500000,1390.900024,1401.500000,1.549958e+09,8.19990,8.24920,8.14060,8.18489
4217,2021-04-30,1399.099976,1405.900024,1396.500000,1397.800049,1.838904e+09,8.20330,8.31147,8.18517,8.18488
4218,2021-05-03,1400.199951,1421.199951,1400.199951,1421.199951,1.946216e+09,8.28770,8.33030,8.24477,8.27455
4219,2021-05-04,1423.199951,1428.099976,1411.000000,1416.300049,2.277663e+09,8.24890,8.34394,8.24491,8.25900


In [17]:
'''
min_max_scaler = preprocessing.MinMaxScaler()
res = min_max_scaler.fit_transform([v for v in X.loc[:, X.columns != "Date"].values])
X.loc[:, X.columns != "Date"] = res
X
'''

'\nmin_max_scaler = preprocessing.MinMaxScaler()\nres = min_max_scaler.fit_transform([v for v in X.loc[:, X.columns != "Date"].values])\nX.loc[:, X.columns != "Date"] = res\nX\n'

In [18]:
# Drop the date columns since we don't use it in training
X = X.drop("Date", 1)
X

,Open_XU_100,High_XU_100,Low_XU_100,Close_XU_100,Volume_XU_100,Open_USD_TRY,High_USD_TRY,Low_USD_TRY,Close_USD_TRY
0,24971.699219,25508.099609,24918.199219,25445.199219,2.278007e+08,1.34550,1.34550,1.33680,1.34400
1,25445.199219,25602.199219,24992.800781,25042.000000,2.688344e+08,1.33680,1.35100,1.33680,1.35100
2,25042.000000,25042.000000,24264.000000,24422.900391,2.062249e+08,1.35450,1.38650,1.35450,1.37400
3,24422.900391,24588.400391,24237.699219,24561.900391,1.618173e+08,1.37800,1.40580,1.37800,1.39250
4,24561.900391,25308.300781,24481.099609,25308.300781,3.037296e+08,1.38700,1.39400,1.36150,1.39400
...,...,...,...,...,...,...,...,...,...
4216,1390.900024,1406.500000,1390.900024,1401.500000,1.549958e+09,8.19990,8.24920,8.14060,8.18489
4217,1399.099976,1405.900024,1396.500000,1397.800049,1.838904e+09,8.20330,8.31147,8.18517,8.18488
4218,1400.199951,1421.199951,1400.199951,1421.199951,1.946216e+09,8.28770,8.33030,8.24477,8.27455
4219,1423.199951,1428.099976,1411.000000,1416.300049,2.277663e+09,8.24890,8.34394,8.24491,8.25900


In [19]:
y_asels = ASELS.drop("Date", 1)
y_asels = y_asels.iloc[:-1,3].reset_index(drop=True)

y_thy = THY.drop("Date", 1)
y_thy = y_thy.iloc[:-1,3].reset_index(drop=True)

y_tuprs = TUPRS.drop("Date", 1)
y_tuprs = y_tuprs.iloc[:-1,3].reset_index(drop=True)

In [20]:
data_full = pd.concat([X, y_asels, y_thy, y_tuprs], axis=1).dropna()

In [21]:
# Subsample
X_sub = data_full.iloc[:100,:-3]
X_sub_valid = data_full.iloc[100:120,:-3]
X_data = data_full.iloc[:4000,:-3]
X_data_valid = data_full.iloc[4000:,:-3]

In [22]:
y_sub = data_full.iloc[:100, -3]
y_sub_valid = data_full.iloc[100:120, -3]
y_data = data_full.iloc[:4000, -3]
y_data_valid = data_full.iloc[4000:, -3]

In [23]:
model = RandomForestRegressor(n_estimators=30, min_samples_leaf=250, max_features=0.5, n_jobs=-1)
%time model.fit(X_sub, y_sub)
model.score(X_sub_valid, y_sub_valid)

Wall time: 252 ms


-1.0261094138562674

In [38]:
model = RandomForestRegressor(n_estimators=30, n_jobs=-1)
%time model.fit(X_data, y_data)
model.score(X_data_valid, y_data_valid)

Wall time: 349 ms


-2.8685977705702923

In [39]:
import matplotlib.pyplot as plt
from sklearn import tree
tree.plot_tree(model.estimators_[0], feature_names=X_data_valid.columns, class_names="target", filled=True)
plt.savefig("test.png")

Error in callback <function flush_figures at 0x000002723136CB80> (for post_execute):


KeyboardInterrupt: 

In [40]:
model.predict(X_data_valid)

array([17.88766647, 17.84233313, 18.0179997 , 17.8203332 , 17.65666643,
       17.6523329 , 18.1190002 , 18.28300047, 18.55200023, 18.49200037,
       18.26066703, 18.28533377, 18.26933383, 18.36500017, 18.404     ,
       18.3930001 , 18.44666667, 18.47733323, 18.51133317, 18.42966667,
       18.51866683, 17.9486663 , 18.1830001 , 17.95433347, 17.9360004 ,
       17.9823335 , 17.99133353, 17.86233307, 17.585333  , 17.54533313,
       17.56233303, 17.61766637, 17.6649998 , 17.6626664 , 17.6473331 ,
       17.67233313, 17.62799963, 17.69566643, 17.81299977, 17.8413332 ,
       17.87266657, 17.7856664 , 17.80666643, 17.6996663 , 17.63999967,
       17.51699977, 17.54266647, 17.58999963, 17.61899967, 17.6106664 ,
       17.52533313, 17.55399973, 17.51766647])

In [41]:
y_data_valid

4167    17.440001
4168    17.350000
4169    17.730000
4170    17.520000
4171    17.080000
4172    17.400000
4173    17.090000
4174    16.980000
4175    17.219999
4176    17.190001
4177    17.209999
4178    17.180000
4179    17.219999
4180    17.030001
4181    17.170000
4182    17.209999
4183    17.139999
4184    17.110001
4185    17.280001
4186    17.240000
4187    17.059999
4188    16.879999
4189    16.370001
4190    14.740000
4191    15.040000
4192    15.290000
4193    15.060000
4194    14.850000
4195    15.120000
4196    15.020000
4197    14.950000
4199    15.500000
4200    15.940000
4201    15.530000
4202    15.610000
4203    15.410000
4204    15.140000
4205    15.050000
4206    15.000000
4207    15.300000
4208    15.140000
4209    15.210000
4210    14.960000
4211    14.920000
4212    14.500000
4213    14.550000
4214    14.670000
4215    15.490000
4216    14.900000
4217    15.020000
4218    15.000000
4219    15.110000
4220    14.910000
Name: Close_ASELS, dtype: float64